In [2]:
import numpy as np 
import random 
import pandas as pd 
import matplotlib.pyplot as plt
from math import *


In [3]:
# Define the alphabet of size 40
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ?!][1234567890"

def generate_random_string(length, alphabet):
    return ''.join(random.choices(alphabet, k=length))

'''mutate letter
   b: letter to be mutated
'''
def mutate(b, alphablet):
    
    action=random.randint(0, 2)
    if action==0:#duplication:
        return b+b
    elif action==1:
        return "" #deletion
    else:
        char_rand=random.randint(0, len(alphablet)-1)
        return alphablet[char_rand]
        
def modify_second_string(B, alphablet):

    B_mutate=""
    for b in B:
        prob=random.random()
        if prob < 0.5: ##mutate:
            B_mutate+=mutate(b, alphablet)
        else:
            B_mutate+=b
            
    return B_mutate
        
#split into segments:
def split_segments(A, B):

    longest_possible_segment = len(A) // 8
    index=0
    list_indices=[index]
    while longest_possible_segment < len(B) - index:
        index=random.randint(index+1, index + longest_possible_segment if index + longest_possible_segment < len(B)  else len(B) - 1)
        list_indices.append(index)
    return list_indices

def select_random_elements(lst, k):
    # Use random.sample to pick k unique elements from the list
    return random.sample(lst, k)
    
def generate_lflcs_instance(n, sigma, alphabet): #instance file: n_sigma_index.txt 

    alphabet=alphabet[:sigma]
    #modify A
    A=generate_random_string(n, alphabet)
    B=A;
    B=modify_second_string(B, alphabet)
    #randomly split B into segments no longer than \A\/8
    points_breaks=split_segments(A, B)
    #print("points_breaks: ", points_breaks)
    #discard >30% segments from points_breaks:
    list_remove_segments=[]
    percentage_to_remove = random.uniform(0.3, 0.7) #random number from 0.3 to 0.7
    #print(percentage_to_remove)
    randomly_select_segments_to_remove=select_random_elements([ i for i in range(len(points_breaks)) ], ceil(percentage_to_remove * len(points_breaks)) ) 
    # Construct M:
    M :str=""
    randomly_select_segments_to_remove.sort()
    #print("randomly_select_segments_to_remove: ", randomly_select_segments_to_remove)

    for seg_remove in randomly_select_segments_to_remove:
        
        #if seg_remove==0:
        #    M+=B[ : points_breaks[0]]
        #else:
        if seg_remove == len(points_breaks)-1:
            M+=B[ points_breaks[seg_remove] : ]
        else:
            M+=B[ points_breaks[seg_remove] : points_breaks[seg_remove+1]]
    # Construct B:
    B_final=""
    remaining_indices=set([ i for i in range(len(points_breaks)) ]).difference(set(randomly_select_segments_to_remove))
    #remaining_indices=list(remaining_indices)
    #remaining_indices=remaining_indices.sort()
    #print("remaining_indices: ", remaining_indices)
    for seg_for_B in remaining_indices:
        #if seg_for_B==0:
        #    B_final+=B[ : points_breaks[0]]
        #    continue
        if seg_for_B == len(points_breaks)-1:
            B_final+=B[ points_breaks[seg_for_B] : ]
        else:
            B_final+=B[ points_breaks[seg_for_B] : points_breaks[seg_for_B + 1] ]     

    #print("B: ", B, len(B))
    return A, B_final, M   


'''save the data into a file ==> create an instance:'''
#DIR_PATH="random_instances/"
def save_into_file(A, B_final, sigma, M, index):

    file_name=DIR_PATH + str(len(A)) + "_" + str(sigma) + "_" + str(index) + ".txt"

    with open(file_name, "w") as f:
        f.write(str(sigma)+" "+str(len(M)) + "\n")
        f.write(A+"\n")
        f.write(B_final+"\n")
        f.write(M)
        

In [4]:
## test
#A, B, M=generate_lflcs_instance(16, 4, ALPHABET)
#print(A, B, len(B),  M, len(M))

In [5]:
## generate the benchmark set:
##we select n = |A| to be 16, 32, 48, 64, 80,
##while the alphabet size is n8 , n4 , n2 . For each combination of
##n and |Σ|, we generate 100 instances
#A_size = [16, 32, 48, 64, 80]
#sigma = [ 0.125, 0.25, 0.5 ]
#number_instances=100

#for asize in A_size:
#    for sig in sigma:
#        for index in range(number_instances):
#            A, B, M = generate_lflcs_instance(asize, int(asize*sig), ALPHABET)
#            save_into_file(A, B, int(asize*sig), M, index)


## Large-sized instance generation

In [6]:
A_size = [600] #[200, 400, 500, 800, 1000]
sigma = [4, 20]
number_instances=10
DIR_PATH="random_larger/"


for asize in A_size:
    for sig in sigma:
        for index in range(number_instances):
            A, B, M = generate_lflcs_instance(asize, sig, ALPHABET)
            save_into_file(A, B, sig, M, index)